In [ ]:
# Install dependencies
%pip install statsmodels seaborn
%pip install tqdm

In [ ]:
import sys
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn

from tbitk.ai.inference_result import *
from pathlib import Path
from tqdm import tqdm

In [ ]:
# This is the only input that will need to be changed
# Point this to the root directory of the model inference results.
INF_RES_ROOT_PATH = Path("data/uos_inference_results_b807a3/")

In [ ]:
HEAD_KEY = {"head_id": ["1", "2", "3"], "min_onsd": [4.3, 5.3, 5.4], "max_onsd": [4.8, 5.6, 5.9]}
HEAD_KEY = pd.DataFrame.from_dict(HEAD_KEY)
HEAD_KEY_MEAN = HEAD_KEY[["head_id"]].copy()
HEAD_KEY_MEAN["head_onsd"]  = (HEAD_KEY["min_onsd"] + HEAD_KEY["max_onsd"]) / 2
print(HEAD_KEY_MEAN)

In [ ]:
PROBE_ABBREV_KEY = {
    "clarius-l7hd": "CL",
    "butterfly-iq": "BF",
    "interson-spl01": "IS",
    "sonoque": "SNQ",
    "sonosite": "SNS"
}

MODEL_ABBREV_KEY = {
    "duke_study_general_small_8c8772": "general",
    "duke_study_clarius_small_8c8772": "clarius",
    "duke_study_butterfly_small_8c8772": "butterfly"
}

def count_frames_with_preds(inference_result):
    # Assumes 3d
    mask = inference_result.mask
    # mask is either a list of 2d itk images or
    # a single 3d itk image
    
    if isinstance(mask, itk.Image):
        mask = itk.array_from_image(mask)
        
    # Now, mask is either a list of 2d itk images
    # or a 3d numpy array

    num_frames_with_eye = 0
    num_frames_with_nerve = 0
    num_frames_with_both = 0
    for frame in mask:
        if isinstance(frame, itk.Image):
            frame = itk.array_from_image(frame)

        if 1 in frame:
            num_frames_with_eye += 1
        if 2 in frame:
            num_frames_with_nerve += 1
        if 1 in frame and 2 in frame:
            num_frames_with_both += 1
            
    return num_frames_with_eye, num_frames_with_nerve, num_frames_with_both
    

def inf_res_to_df(result_paths):
    # Build dataframe
    df = pd.DataFrame(columns=(
        "probe",
        "model_name",
        "participant_id",
        "head_id",
        "model_onsd",
        "n_frames_with_eye",
        "n_frames_with_nerve",
        "n_frames_with_both",
        "frac_frames_with_eye",
        "frac_frames_with_nerve",
        "frac_frames_with_both"
    ))
    for p in tqdm(list(result_paths)):
        res = InferenceResult3D.load_from_dir(p)

        # Now extract data from pathname
        split_name = p.name.split("-") # 2-1-3 -> [2, 1, 3]
        participant_id = split_name[0]
        head_id = split_name[1]
        
        # And the parent directories
        probe = PROBE_ABBREV_KEY[p.parents[2].name]
        model_name = MODEL_ABBREV_KEY[p.parents[1].name]

        n_frames_with_eye, n_frames_with_nerve, n_frames_with_both = count_frames_with_preds(res)
        n_frames = len(res)

        frac_frames_with_eye = n_frames_with_eye / n_frames
        frac_frames_with_nerve = n_frames_with_nerve / n_frames
        frac_frames_with_both = n_frames_with_both / n_frames
        df.loc[len(df)] = [
            probe,
            model_name,
            participant_id,
            head_id,
            res.onsd,
            n_frames_with_eye,
            n_frames_with_nerve,
            n_frames_with_both,
            frac_frames_with_eye,
            frac_frames_with_nerve,
            frac_frames_with_both
        ]

    return df

In [ ]:
model_preds_df = inf_res_to_df(INF_RES_ROOT_PATH.glob("**/*_inf_res"))

In [ ]:
# Merge with head key and do some other type conversions
model_preds_df.model_onsd = model_preds_df.model_onsd.astype("float64")
model_preds_df = model_preds_df.merge(HEAD_KEY_MEAN, on="head_id")

In [ ]:
# Set up error ranges
model_preds_df["abs_error"] = np.abs(model_preds_df["model_onsd"] - model_preds_df["head_onsd"])

In [ ]:
model_preds_df

In [ ]:
model_preds_df["modelXprobe"] = model_preds_df["model_name"] + "X" + model_preds_df["probe"]
ax = seaborn.stripplot(data=model_preds_df, x="head_id", y="model_onsd", hue="modelXprobe", dodge=True)
ax.figure.savefig("data/figures/model_onsd_per_head.jpeg")

In [ ]:
ax = seaborn.stripplot(data=model_preds_df, x="head_id", y="frac_frames_with_eye", hue="modelXprobe", dodge=True)
ax.figure.savefig("data/figures/frac_frames_with_eye_per_head.jpeg")

In [ ]:
ax = seaborn.stripplot(data=model_preds_df, x="head_id", y="frac_frames_with_nerve", hue="modelXprobe", dodge=True)
ax.figure.savefig("data/figures/frac_frames_with_nerve_per_head.jpeg")

In [ ]:
ax = seaborn.stripplot(data=model_preds_df, x="participant_id", y="model_onsd", hue="modelXprobe", dodge=True)
ax.figure.savefig("data/figures/model_onsd_per_participant.jpeg")

In [ ]:
ax = seaborn.stripplot(data=model_preds_df, x="participant_id", y="abs_error", hue="modelXprobe", dodge=True)
ax.figure.savefig("data/figures/abs_error_per_participant.jpeg")